In [1]:
import numpy as np
import pandas as pd
from collections import Counter

tol = 1e-3

In [2]:
setup = pd.read_csv("../data/dispenser_setup.csv", comment='#')

In [3]:
Polymers = ["PLA"]

In [4]:
relevant_component = dict()
matrix = dict()
additive = dict()

for index, row in setup.iterrows():
    non_zero_elements = [(col, value) for col, value in row.items() if (value != 0 and col != "Dispenser")]
    if non_zero_elements:
        dispenser_string = "Dispenser " + str(index+1) + ": "
        for element in non_zero_elements:
            dispenser_string += str(element[1]*100) + "% of " + element[0] + ", "
            if element[0] in Polymers:
                matrix[index+1] = element[1]
            else:
                additive[index+1] = element[1]
        if len(non_zero_elements) == 1:
            if non_zero_elements[0][0] in Polymers:
                relevant_component[index+1] = non_zero_elements[0][0]
            else:
                print("Error, polymer in dispenser ", index, " not specified")
                input("Press Enter to continue")
                
        elif len(non_zero_elements) == 2:
            relevant_component[index+1] = [element[0] for element in non_zero_elements if element[0] not in Polymers][0]
        else:
            print("The material in dispenser ", index, " contains more than 2 additives")
            input("Press Enter to continue")
        print(dispenser_string)
        
        total = np.sum([element[1] for element in non_zero_elements])
        if np.abs(total-1) > tol:
            print("Error in composition in dispenser ", index, ", check if the setup file is correct")
            input("Press Enter to continue")

value_counts = Counter(relevant_component.values())
repeated_values = [value for value, count in value_counts.items() if count > 1]
if repeated_values:
    print("There are repeated values in your setup, we recommend you to bridge those dispensers together for increased flow. Otherwise, only one of them will be used")
    input("Press Enter to continue")
    
dispensers_map =  {value: key for key, value in relevant_component.items()}

Dispenser 4: 100.0% of PLA, 
Dispenser 5: 90.0% of PLA, 10.0% of GNP, 
Dispenser 6: 90.0% of PLA, 10.0% of BN, 


In [5]:
M = np.zeros(8)
A = np.zeros(8)

for i in range(len(M)):
    if i+1 in matrix.keys():
        M[i] = matrix[i+1]
    if i+1 in additive.keys():
        A[i] = additive[i+1]

In [6]:
compositions = pd.read_csv("../data/compositions_to_dispense.csv", comment='#')

for index, row in compositions.iterrows():
    non_zero_elements = [(col, value) for col, value in row.items() if (value != 0 and col != "Composition")]
    X = np.zeros(8)
    for element in non_zero_elements:
        X[dispensers_map[element[0]]-1] = element[1]
    C = np.zeros(8)
    polymer = dispensers_map[Polymers[0]]
    for i in range(len(C)):
        if A[i] > tol:
            C[i] = X[i]/A[i]
    C[polymer-1] = X[polymer-1] - np.sum(C * M)
    if 1-np.sum(X) < tol:
        print('Concentrations add up.')
    else:
        print('WARNING: Concentrations do not add up.')
        print('**************************************')
        print('Check user input before continuing.')

Concentrations add up.


In [7]:
C

array([0. , 0. , 0. , 0.5, 0.2, 0.3, 0. , 0. ])

In [8]:
len(setup)

8